![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step and it's important to keep a close eye on the process.

For this project, you're supporting a team that wants to improve the way they're monitoring and controlling a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process is working well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

Using SQL window functions, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [184]:
-- Write your query here
select * 
from manufacturing_parts

,item_no,length,width,height,operator
0,1,102.67,49.53,19.69,Op-1
1,2,102.50,51.42,19.63,Op-1
2,3,95.37,52.25,21.51,Op-1
3,4,94.77,49.24,18.60,Op-1
4,5,104.26,47.90,19.46,Op-1
...,...,...,...,...,...
495,496,101.24,49.03,20.96,Op-20
496,497,98.37,52.12,19.68,Op-20
497,498,96.49,48.78,19.19,Op-20
498,499,94.16,48.39,21.60,Op-20


In [185]:
select operator,
ROW_NUMBER() over my_window as row_number,
height,
AVG(height) over my_window as avg_height,
STDDEV(height) over my_window as stddev_height
from manufacturing_parts
WINDOW  my_window as (PARTITION BY operator ORDER BY item_no DESC ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)

,operator,row_number,height,avg_height,stddev_height
0,Op-1,1,21.22,21.22,NaN
1,Op-1,2,19.54,20.38,1.19
2,Op-1,3,20.17,20.31,0.85
3,Op-1,4,22.31,20.81,1.22
4,Op-1,5,19.79,20.61,1.15
...,...,...,...,...,...
495,Op-9,31,19.19,19.79,0.90
496,Op-9,32,21.90,20.17,1.32
497,Op-9,33,19.68,19.96,1.29
498,Op-9,34,21.40,20.53,1.14


In [186]:
select a.* , avg_height + (3 * (stddev_height / sqrt(5))) as ucl ,
		avg_height -  (3 * (stddev_height / sqrt(5))) as lcl 
from 
(
select operator,
ROW_NUMBER() over my_window as row_number,
height,
round((AVG(height) over my_window )::numeric,2)as avg_height,
round((STDDEV(height) over my_window )::numeric,2 )as stddev_height
from manufacturing_parts
WINDOW  my_window as (PARTITION BY operator ORDER BY item_no DESC ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)
) as a;

,operator,row_number,height,avg_height,stddev_height,ucl,lcl
0,Op-1,1,21.22,21.22,NaN,NaN,NaN
1,Op-1,2,19.54,20.38,1.19,21.98,18.78
2,Op-1,3,20.17,20.31,0.85,21.45,19.17
3,Op-1,4,22.31,20.81,1.22,22.45,19.17
4,Op-1,5,19.79,20.61,1.15,22.15,19.07
...,...,...,...,...,...,...,...
495,Op-9,31,19.19,19.79,0.90,21.00,18.58
496,Op-9,32,21.90,20.17,1.32,21.94,18.40
497,Op-9,33,19.68,19.96,1.29,21.69,18.23
498,Op-9,34,21.40,20.53,1.14,22.06,19.00


In [187]:
select b.* , case when height between lcl and ucl then False else True end as alert
from 
(
select a.* ,avg_height + (3 * (stddev_height / sqrt(5))) as ucl ,
		avg_height -  (3 * (stddev_height / sqrt(5))) as lcl 
from 
(
select operator,
ROW_NUMBER() over my_window as row_number,
height,
AVG(height) over my_window as avg_height,
STDDEV(height) over my_window as stddev_height
from manufacturing_parts
WINDOW  my_window as (PARTITION BY operator ORDER BY item_no ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)
) as a
) as b
WHERE row_number >= 5;

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-1,5,19.46,19.778,1.062812,21.203912,18.352088,False
1,Op-1,6,20.36,19.912,1.090812,21.375477,18.448523,False
2,Op-1,7,20.22,20.030,1.084574,21.485108,18.574892,False
3,Op-1,8,21.03,19.934,0.931225,21.183369,18.684631,False
4,Op-1,9,19.78,20.170,0.598832,20.973418,19.366582,False
...,...,...,...,...,...,...,...,...
415,Op-9,31,19.01,18.904,0.203052,19.176422,18.631578,False
416,Op-9,32,18.57,18.864,0.250260,19.199759,18.528241,False
417,Op-9,33,20.91,19.266,0.952276,20.543613,17.988387,True
418,Op-9,34,21.24,19.678,1.291112,21.410208,17.945792,False
